## Автоформируемый DDL и Truncate, сверка таблиц и работа с графами

### Импорт библиотек

In [1]:
from IPython.display import FileLink
import sql_excel_processing as sep
from impala.dbapi import connect

### Вводимые вручную данные

In [3]:
# БД, в которой таблица находится
db_name = 'sbx_041'

# Наименование таблицы
table_name = 'bvg_target_response'

# Постфикс (обычно либо «_tmp», либо отсутствует)
postfix = ''

# Убрать из наименования таблиц
skip_in_queries = ['']

# Загружаемый файл запроса
sql_file = 'table/bvg_target_response.sql'

# Загружаемый Excel-файл со структурой целевой таблицы
excel_file = 'Поля таблицы BVG_TARGET_RESPONSE v2.xlsx'

В случае возникновения ошибки `«ticket expired»` в процессе выполнения кода, необходимо получить новый тикет доступа в конце ноутбука.

#### Изменённый запрос

In [4]:
query = sep.query_replace(sql_file, table_name, skip_in_queries)
FileLink(table_name + '.sql')

/work/sql_excel_processing/bvg_target_response.sql

#### Получение DDL целевой таблицы

In [5]:
ddl_query = sep.auto_ddl(db_name, table_name, excel_file, skip_in_queries)
FileLink('query_ddl.sql')

/work/sql_excel_processing/query_ddl.sql

#### Получение DDL временных таблиц

In [6]:
ddl_temp_query = sep.auto_temp_ddl(db_name, table_name, postfix, sql_file, 'файл', skip_in_queries)
FileLink('query_temp_ddl.sql')

/work/sql_excel_processing/query_temp_ddl.sql

#### Получение Truncate запросов временных таблиц

In [7]:
query_truncate = sep.auto_truncate(db_name, table_name, postfix, sql_file, 'файл', skip_in_queries)
FileLink('query_truncate.sql')

/work/sql_excel_processing/query_truncate.sql

#### Получение Excel-файла с наименованиями, типами данных полей и комментариями из Impala

In [8]:
sep.auto_table(db_name, table_name)
FileLink('structure.xlsx')

/work/sql_excel_processing/structure.xlsx

#### Получение отчёта по результатам сверки структуры таблицы и описания в Confluence

In [9]:
sep.structure_compare(db_name, table_name, excel_file)
FileLink('diff_report.xlsx')

/work/sql_excel_processing/diff_report.xlsx

#### Автоматическое построение графа взаимосвязи таблиц из SQL-запроса

In [10]:
er_table_tmp_df = sep.get_child_parent(sql_file, skip_in_queries)
sep.draw_graph(er_table_tmp_df)
FileLink('graph.pdf')

Количество уникальных связей child | parent: 61


/work/sql_excel_processing/graph.pdf

#### Извлечение зависимостей "таблица – источник" из SQL-запроса

In [11]:
er_table_tmp_df

,child,parent
0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_global_individual
0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_x_global
0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_global_identity_card
1,sbx_dml.bvg_target_response_act_m_tmp,dm_evd.agg_party_global_activity
2,sbx_dml.bvg_target_response_phones_tmp,dm_evd.xref_calendar
...,...,...
14,sbx_dml.bvg_target_response_comm_t1_tmp,dm_rto_etl_view.cdm_ma_offer
14,sbx_dml.bvg_target_response_comm_t1_tmp,dm_evd.party_global_individual
15,sbx_041.bvg_target_response,sbx_dml.bvg_target_response_cln_attr_tmp
15,sbx_041.bvg_target_response,sbx_dml.bvg_target_response_comm_t1_tmp


In [12]:
er_table_tmp_df = sep.get_child_parent(sql_file, skip_in_queries)
er_table_df = sep.table_dependencies(er_table_tmp_df)
er_table_df

Количество уникальных связей final | child | parent: 106


,final,index,child,parent,changed_dttm
0,sbx_041.bvg_target_response,0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_global_individual,2022-03-24 16:53:56
1,sbx_041.bvg_target_response,0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_x_global,2022-03-24 16:53:56
2,sbx_041.bvg_target_response,0,sbx_dml.bvg_target_response_cln_tmp,dm_evd.party_global_identity_card,2022-03-24 16:53:56
3,sbx_041.bvg_target_response,1,sbx_dml.bvg_target_response_act_m_tmp,dm_evd.agg_party_global_activity,2022-03-24 16:53:56
4,sbx_041.bvg_target_response,2,sbx_dml.bvg_target_response_phones_tmp,dm_evd.xref_calendar,2022-03-24 16:53:56
...,...,...,...,...,...
101,sbx_041.bvg_target_response,15,sbx_041.bvg_target_response,dm_evd.card_bank,2022-03-24 16:53:56
102,sbx_041.bvg_target_response,15,sbx_041.bvg_target_response,dm_evd.agreement_loan,2022-03-24 16:53:56
103,sbx_041.bvg_target_response,15,sbx_041.bvg_target_response,stock.insurance_sogazlife,2022-03-24 16:53:56
104,sbx_041.bvg_target_response,15,sbx_041.bvg_target_response,dm_evd.t_party_info_x_global,2022-03-24 16:53:56


##### Внесение в таблицу вышеуказанного DataFrame "таблица – источник"

In [21]:
# Перенос данных в таблицу sbx_041.pde_table_dependencies
conn = connect(host = 'hdp-p3pml', port = 21050, use_ssl = 'true', auth_mechanism = 'GSSAPI')
cursor = conn.cursor()

try:
    for index, row in er_table_df.iterrows():
        cursor.execute("INSERT INTO sbx_041.pde_table_dependencies SELECT '" + row['final'] + "', " + str(row['index'] + 1) + ", '" + 
                       row['child'] + "', '" + row['parent'] + "', '" + row['changed_dttm'] + "'")
    print("Данные перенесены.")

except:
    print("Данные не перенесены.")

Данные перенесены.


#### Поиск неиспользуемых таблиц и полей в SQL-запросе

In [13]:
er_table_tmp_df = sep.get_child_parent(sql_file, skip_in_queries)
sep.unused_tables_fields(sql_file, er_table_tmp_df)

Анализ скрипта витрины sbx_041.bvg_target_response

Количество таблиц с неиспользуемыми полями: 7, количество неиспользуемых полей: 44

Неиспользуемые поля для таблицы sbx_dml.bvg_target_response_all_app_tmp:
   • amount_rur
   • app_id
   • app_source
   • open_dt
   • request_rk
   • root_agreement_rk

Неиспользуемые поля для таблицы sbx_dml.bvg_target_response_cln_attr_tmp:
   • birth_dt
   • cl_category_pro
   • first_name
   • last_name
   • middle_name
   • vip_flg

Неиспользуемые поля для таблицы sbx_dml.bvg_target_response_comm_t1_tmp:
   • channel_nm
   • comm_campaign_name
   • communication_type
   • contact_dttm
   • contact_history_status_cd
   • contact_history_status_desc
   • contact_id
   • message_text
   • mo_product_cd
   • participant_id
   • phone1
   • product_name
   • segment_name

Неиспользуемые поля для таблицы sbx_dml.bvg_target_response_ins_prod_tmp:
   • birth_dt
   • city
   • insprogram
   • insurance_company
   • insurancefee
   • party_info_desc
   • p

#### Форматирование SQL-запроса

In [14]:
sep.sql_formatter(sql_file)
FileLink('formatted_query.sql')

/work/sql_excel_processing/formatted_query.sql

#### Получение нового ticket-а доступа в случае возникновения ошибки `ticket expired`

In [4]:
import os

login = 'gpbu16371'        # Ваш логин
password = '***'           # Ваш пароль
!klist

os.system('echo ' + password + ' | kinit ' + login)
!klist

Ticket cache: FILE:/tmp/krb5cc
Default principal: gpbu16371@INT.GAZPROMBANK.RU

Valid starting       Expires              Service principal
01/11/2022 15:54:09  01/12/2022 01:54:09  krbtgt/INT.GAZPROMBANK.RU@INT.GAZPROMBANK.RU
	renew until 01/17/2022 14:53:05
Ticket cache: FILE:/tmp/krb5cc
Default principal: gpbu16371@INT.GAZPROMBANK.RU

Valid starting       Expires              Service principal
01/11/2022 15:54:09  01/12/2022 01:54:09  krbtgt/INT.GAZPROMBANK.RU@INT.GAZPROMBANK.RU
	renew until 01/17/2022 14:53:05
